# Trabalho 3 - Mineração de Dados - DCC127
Professor: Heder Soares Bernardino 

Lucas Augusto Muller
Pedro Henrique Muniz

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/lucasmullers/mineracao-2020.3.git

Cloning into 'mineracao-2020.3'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.


In [ ]:
import pandas as pd
import numpy as np
import IPython.display
import sys
from scipy import stats

# Leitura dos dados

Inicialmente as duas bases de dados são lidas e o tipo de cada atributo é impresso.

In [ ]:
dadosTrab2 = pd.read_csv('/content/mineracao-2020.3/trabalho2_dados_5.csv')

display(dadosTrab2[:5])
print("Tipos dos dados (DADOS TRAB2): \n{}\n".format(dadosTrab2.dtypes))

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Legend of the Dragon,Wii,Fighting,Game Factory,0.06,1-May-07,37.0,14.0,4.8,5.0,Neko Entertainment
1,Kawashima Ryuuta Kyouju Kanshuu Nouryoku Train...,PSP,Misc,Sega,0.09,NaN,NaN,NaN,NaN,NaN,NaN
2,Backyard NFL Football '10,X360,Sports,Atari,0.10,NaN,NaN,NaN,NaN,NaN,NaN
3,Payday 2,PS3,Action,505 Games,0.22,13-Aug-13,74.0,12.0,7.2,106.0,Overkill Software
4,NBA 2K11,Wii,Sports,Take-Two Interactive,0.47,19-Oct-10,NaN,NaN,tbd,NaN,Visual Concepts


Tipos dos dados (DADOS TRAB2): 
nome                   object
plataforma             object
genero                 object
editora                object
vendas                float64
lancamento             object
avaliacao-criticos    float64
numero-criticos       float64
avaliacao-usuarios     object
numero-usuarios       float64
fabricante             object
dtype: object



In [ ]:
dadosTrab3 = pd.read_csv('/content/mineracao-2020.3/trabalho3_dados_adicionais_5.csv')

display(dadosTrab3[:5])
print("Tipos dos dados (DADOS TRAB3): \n{}\n".format(dadosTrab3.dtypes))

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Ridge Racer,PSP,Racing,Sony Computer Entertainment,1.39,22-Mar-05,88.0,58.0,8.2,57.0,Namco
1,Tiger Woods PGA Tour 06,X360,Sports,Electronic Arts,0.30,16-Nov-05,71.0,36.0,7.4,8.0,EA Sports
2,Peter Jackson's King Kong: The Official Game o...,X360,Action,Ubisoft,0.23,21-Nov-05,80.0,57.0,7.2,60.0,Ubisoft Montpellier
3,Tiger Woods PGA Tour 06,PSP,Sports,Electronic Arts,0.20,26-Sep-05,80.0,15.0,9,7.0,Team Fusion
4,Peter Jackson's King Kong: The Official Game o...,PSP,Action,Ubisoft,0.18,20-Dec-05,56.0,10.0,6.2,20.0,Phoenix Games Studio


Tipos dos dados (DADOS TRAB3): 
nome                   object
plataforma             object
genero                 object
editora                object
vendas                float64
lancamento             object
avaliacao-criticos    float64
numero-criticos       float64
avaliacao-usuarios     object
numero-usuarios       float64
fabricante             object
dtype: object



# Concatenação das bases

Verificando as colunas e os "tipos" de dados nós consideramos que os dois arquivos possuem dados de uma mesma base, portanto nós as integramos sem necessitar tratar inconsistências nos formatos dos atributos.

In [ ]:
dados = pd.concat([dadosTrab2, dadosTrab3], axis=0, ignore_index=True)
display(dados)

# dados['fabricante'] = dados['fabricante'].replace('Rovio Mobile', 'Rovio Entertainment Ltd')
# dados['fabricante'] = dados['fabricante'].replace('Ubisoft Quebec', 'Ubisoft')

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Legend of the Dragon,Wii,Fighting,Game Factory,0.06,1-May-07,37.0,14.0,4.8,5.0,Neko Entertainment
1,Kawashima Ryuuta Kyouju Kanshuu Nouryoku Train...,PSP,Misc,Sega,0.09,NaN,NaN,NaN,NaN,NaN,NaN
2,Backyard NFL Football '10,X360,Sports,Atari,0.10,NaN,NaN,NaN,NaN,NaN,NaN
3,Payday 2,PS3,Action,505 Games,0.22,13-Aug-13,74.0,12.0,7.2,106.0,Overkill Software
4,NBA 2K11,Wii,Sports,Take-Two Interactive,0.47,19-Oct-10,NaN,NaN,tbd,NaN,Visual Concepts
...,...,...,...,...,...,...,...,...,...,...,...
3209,Major League Baseball 2K8,PSP,Sports,Unknown,0.03,3-Mar-08,63.0,5.0,tbd,NaN,Kush Games
3210,The History Channel: Great Battles - Medieval,PS3,Strategy,Slitherine Software,0.03,NaN,NaN,NaN,NaN,NaN,NaN
3211,Battle vs. Chess,PS3,Misc,TopWare Interactive,0.03,TBA,NaN,NaN,NaN,NaN,TopWare Interactive
3212,Dance! It's Your Stage,Wii,Misc,DTP Entertainment,0.01,NaN,NaN,NaN,NaN,NaN,NaN


A seguir, são detectadas e tratadas inconsistências de dados fora do formato padrão na base. Os dados que possuíam a informação do ano foram substituídos por "01-Jan-ANO", pois nós decidimos trabalhar somente com o valor do ano. Além disso, o valor "tbd" foi substituido por NaN na coluna 'avaliacao-usuarios'.

*   Remoção das linhas que possuem 'Canceled' na coluna lancamento
*   Substituição do valor 'TBA' por 'NaN' na coluna lancamento
*   Substiuição de valores com ano por "01-Jan-ANO"
*   Conversão do formato da data de lançamento para obter o ano
*   Substituição do valor 'tbd' por 'NaN' na coluna avaliacao-usuario


In [ ]:
dadosLancamento = dados[~dados.lancamento.str.contains('(\d{1})[-](\D{3})[-](\d{2})', na=False)]
dadosLancamento = dadosLancamento.lancamento.dropna()
print(dadosLancamento.unique())

dadosAvaliacao = dados[~dados['avaliacao-usuarios'].str.contains('((\d{1})[.](\d{1}))', na=False)]
dadosAvaliacao = dadosAvaliacao['avaliacao-usuarios'].dropna()
print(dadosAvaliacao.unique())

['TBA' 'Q2 2014' 'Canceled' '2007' 'TBA 2014' 'Jun-10' 'TBA 2013'
 'Q2 2011']
['tbd' '9' '6' '7' '5' '8' '4' '3' '2']


/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [ ]:
dados = dados[dados['lancamento'] != 'Canceled']
dados['lancamento'] = dados['lancamento'].replace('TBA', float('NaN'))
dados['lancamento'] = dados['lancamento'].replace('Q2 2014', '01-Jan-14')
dados['lancamento'] = dados['lancamento'].replace('2007', '01-Jan-07')
dados['lancamento'] = dados['lancamento'].replace('TBA 2014', '01-Jan-14')
dados['lancamento'] = dados['lancamento'].replace('Jun-10', '01-Jun-10')
dados['lancamento'] = dados['lancamento'].replace('TBA 2013', '01-Jan-13')
dados['lancamento'] = dados['lancamento'].replace('Q2 2011', '01-Jan-11')
dados['lancamento'] = pd.to_datetime(dados['lancamento'], format='%d-%b-%y', errors = 'coerce')
dados['anoLancamento'] = dados['lancamento'].dt.year
dados = dados.drop(['lancamento'], axis=1)

dados['avaliacao-usuarios'] = dados['avaliacao-usuarios'].replace('tbd', 'nan').astype('float64')
display(dados)
print("Quantidade de NaN por coluna: \n{}\n".format(dados.isnull().sum()))

,nome,plataforma,genero,editora,vendas,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante,anoLancamento
0,Legend of the Dragon,Wii,Fighting,Game Factory,0.06,37.0,14.0,4.8,5.0,Neko Entertainment,2007.0
1,Kawashima Ryuuta Kyouju Kanshuu Nouryoku Train...,PSP,Misc,Sega,0.09,NaN,NaN,NaN,NaN,NaN,NaN
2,Backyard NFL Football '10,X360,Sports,Atari,0.10,NaN,NaN,NaN,NaN,NaN,NaN
3,Payday 2,PS3,Action,505 Games,0.22,74.0,12.0,7.2,106.0,Overkill Software,2013.0
4,NBA 2K11,Wii,Sports,Take-Two Interactive,0.47,NaN,NaN,NaN,NaN,Visual Concepts,2010.0
...,...,...,...,...,...,...,...,...,...,...,...
3209,Major League Baseball 2K8,PSP,Sports,Unknown,0.03,63.0,5.0,NaN,NaN,Kush Games,2008.0
3210,The History Channel: Great Battles - Medieval,PS3,Strategy,Slitherine Software,0.03,NaN,NaN,NaN,NaN,NaN,NaN
3211,Battle vs. Chess,PS3,Misc,TopWare Interactive,0.03,NaN,NaN,NaN,NaN,TopWare Interactive,NaN
3212,Dance! It's Your Stage,Wii,Misc,DTP Entertainment,0.01,NaN,NaN,NaN,NaN,NaN,NaN


Quantidade de NaN por coluna: 
nome                     0
plataforma               0
genero                   0
editora                  5
vendas                   0
avaliacao-criticos    1151
numero-criticos       1151
avaliacao-usuarios    1252
numero-usuarios       1252
fabricante             704
anoLancamento          707
dtype: int64



Uma contagem de dados vazios de cada atributo foi realizada, bem como a quantidade de objetos duplicados.

# Detecção e remoção de objetos repetidos

Objetos repetidos foram descartados da base de dados.

In [ ]:
print("Quantidade de valores duplicados: {}".format(dados.duplicated().sum()))
duplicatedDFRow = dados[dados.duplicated(keep=False)]
duplicatedDFRow = duplicatedDFRow.sort_values(by=['nome'])
display(duplicatedDFRow)

Quantidade de valores duplicados: 799


,nome,plataforma,genero,editora,vendas,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante,anoLancamento
2260,11eyes: CrossOver,X360,Adventure,5pb,0.02,NaN,NaN,NaN,NaN,NaN,NaN
745,11eyes: CrossOver,X360,Adventure,5pb,0.02,NaN,NaN,NaN,NaN,NaN,NaN
238,AKB1/149: Love Election,PS3,Adventure,Namco Bandai Games,0.06,NaN,NaN,NaN,NaN,NaN,NaN
3114,AKB1/149: Love Election,PS3,Adventure,Namco Bandai Games,0.06,NaN,NaN,NaN,NaN,NaN,NaN
2582,AKB1/48: Idol to Guam de Koishitara...,PSP,Misc,Namco Bandai Games,0.40,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1219,de Blob 2,Wii,Platform,THQ,0.16,79.0,20.0,8.0,23.0,Blue Tongue Entertainment,2011.0
2463,nail'd,PS3,Racing,Deep Silver,0.08,66.0,36.0,6.5,10.0,Techland,2010.0
1365,nail'd,PS3,Racing,Deep Silver,0.08,66.0,36.0,6.5,10.0,Techland,2010.0
124,pro evolution soccer 2011,PS3,Sports,Konami Digital Entertainment,2.38,77.0,41.0,6.7,70.0,Konami,2010.0


In [ ]:
dados = dados.drop_duplicates()
# dados.sort_values('nome').to_csv('/content/drive/MyDrive/Mineração de Dados/wth_duplicated.csv')
print("Quantidade de valores duplicados: {}".format(dados.duplicated().sum()))
display(dados)

Quantidade de valores duplicados: 0


,nome,plataforma,genero,editora,vendas,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante,anoLancamento
0,Legend of the Dragon,Wii,Fighting,Game Factory,0.06,37.0,14.0,4.8,5.0,Neko Entertainment,2007.0
1,Kawashima Ryuuta Kyouju Kanshuu Nouryoku Train...,PSP,Misc,Sega,0.09,NaN,NaN,NaN,NaN,NaN,NaN
2,Backyard NFL Football '10,X360,Sports,Atari,0.10,NaN,NaN,NaN,NaN,NaN,NaN
3,Payday 2,PS3,Action,505 Games,0.22,74.0,12.0,7.2,106.0,Overkill Software,2013.0
4,NBA 2K11,Wii,Sports,Take-Two Interactive,0.47,NaN,NaN,NaN,NaN,Visual Concepts,2010.0
...,...,...,...,...,...,...,...,...,...,...,...
3205,Get Fit with Mel B,X360,Sports,Black Bean Games,0.07,57.0,7.0,NaN,NaN,Lightning Fish Games,2011.0
3206,Captain America: Super Soldier,Wii,Action,Sega,0.05,57.0,16.0,6.4,5.0,High Voltage Software,2011.0
3207,Port Royale 3,PS3,Simulation,Kalypso Media,0.04,NaN,NaN,8.5,4.0,Kalypso,2012.0
3210,The History Channel: Great Battles - Medieval,PS3,Strategy,Slitherine Software,0.03,NaN,NaN,NaN,NaN,NaN,NaN


# Tratamento de jogos com nomes repetidos

É feito então o tratamento das colunas, de forma a unificar as linhas que representam o mesmo jogo. 

No caso, o nome do jogo foi mantido o mesmo. Nas colunas relativas ao número de vendas, número de críticos e número de usuários, foi inserida a soma dos valores. Para as colunas avaliação dos críticos e avaliação dos usuários, foi inserida a média das avaliações. No caso do ano de lançamento, foi escolhida a moda, e finalmente, as plataformas, editoras, gêneros e fabricantes que possuíam diferentes valores únicos foram concatenados. A nova base de dados pode ser vista abaixo.


In [ ]:
duplicatedNames = dados[dados.duplicated(subset=['nome'], keep=False)]
duplicatedNames = duplicatedNames.sort_values(by=['nome'])
# display(duplicatedNames)

names = duplicatedNames['nome'].unique()
tempDf = pd.DataFrame(columns=dados.columns)

for name in names:
    finalRow = []
    for index, col in enumerate(dados.columns):
        if col == 'nome':
            duplicatedRows = duplicatedNames[duplicatedNames['nome'] == name]
            finalRow.append(duplicatedRows.iloc[0, index])
        elif col == 'vendas' or col == 'numero-criticos' or col == 'numero-usuarios':
            finalRow.append(duplicatedRows[col].sum())
        elif col == 'avaliacao-criticos' or col == 'avaliacao-usuarios':
            finalRow.append(duplicatedRows[col].mean())
        elif col == 'anoLancamento':
            if(len(duplicatedRows[col].dropna().unique()) != 1):
                # print(duplicatedRows[col].mode())
                finalRow.append(stats.mode(duplicatedRows[col])[0])
            else:
                finalRow.append(duplicatedRows.iloc[0,index]) # Fabricante
        else:
            if(len(duplicatedRows[col].dropna().unique()) == 0):
                finalRow.append(float('nan'))
            elif(len(duplicatedRows[col].dropna().unique()) != 1):
                concat = ''
                uniqueList = np.sort(duplicatedRows[col].dropna().unique())
                for i in range(len(uniqueList)):
                    if(i == len(uniqueList) - 1):
                        concat += uniqueList[i]
                    else:
                        concat += uniqueList[i] + ', '
                finalRow.append(concat)
            else:
                finalRow.append(duplicatedRows.iloc[0, index])

    tempDf.loc[len(tempDf)] = finalRow

dados = dados.drop_duplicates(subset=['nome'], keep=False)
# print(dados)
dados = pd.concat([dados, tempDf], axis=0, ignore_index=True)
# dados.sort_values('nome').to_csv('/content/drive/MyDrive/Mineração de Dados/wth_inconsistencias.csv')
display(dados)

,nome,plataforma,genero,editora,vendas,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante,anoLancamento
0,Kawashima Ryuuta Kyouju Kanshuu Nouryoku Train...,PSP,Misc,Sega,0.09,NaN,NaN,NaN,NaN,NaN,NaN
1,DT Carnage,PSP,Racing,Agetec,0.04,NaN,NaN,NaN,NaN,Axis Entertainment,2009
2,DeathSmiles,X360,Shooter,Rising Star Games,0.25,76.000000,29.0,8.50,41.0,Cave,2010
3,Muv-Luv Alternative,PS3,Adventure,5pb,0.01,NaN,NaN,NaN,NaN,NaN,NaN
4,MLB,PSP,Sports,Sony Computer Entertainment,0.24,80.000000,26.0,8.70,15.0,989 Sports,2005
...,...,...,...,...,...,...,...,...,...,...,...
1412,Zumba Fitness,"PS3, Wii, X360",Sports,505 Games,9.20,42.000000,10.0,6.00,13.0,"Pipeworks Software, Inc.",2010
1413,Zumba Fitness: World Party,"Wii, XOne",Misc,Majesco Entertainment,0.15,73.000000,5.0,6.60,31.0,Zoe Mode,2013
1414,de Blob 2,"PS3, Wii, X360",Platform,THQ,0.43,76.666667,124.0,7.50,69.0,Blue Tongue Entertainment,2011
1415,nail'd,"PS3, X360",Racing,Deep Silver,0.16,65.000000,85.0,7.15,22.0,Techland,2010


# Tratamento de valores faltantes

Com os dados já unificados (por nome do jogo), foi realizada uma nova contagem dos valores vazios ('nans') e como apenas 5 das linhas não possuiam editora, optou-se por removê-las da base.

In [ ]:
print("Quantidade de NaNs no dataset: \n{}".format(dados.isnull().sum()))
print(dados)
dados = dados.dropna(subset=['editora'])

Quantidade de NaNs no dataset: 
nome                    0
plataforma              0
genero                  0
editora                 5
vendas                  0
avaliacao-criticos    625
numero-criticos       501
avaliacao-usuarios    658
numero-usuarios       527
fabricante            462
anoLancamento         379
dtype: int64
                                                   nome  ... anoLancamento
0     Kawashima Ryuuta Kyouju Kanshuu Nouryoku Train...  ...           NaN
1                                            DT Carnage  ...          2009
2                                           DeathSmiles  ...          2010
3                                   Muv-Luv Alternative  ...           NaN
4                                                   MLB  ...          2005
...                                                 ...  ...           ...
1412                                      Zumba Fitness  ...          2010
1413                         Zumba Fitness: World Party  ...         

Foi então tratado os valores vazios das colunas numéricas. Utilizou-se o SimpleImputer do ScikitLearn, e os valores vazios foram substituídos pela mediana do respectivo atributo. Dessa forma, imprimindo novamente os valores vazios da  tabela, nota-se que restaram apenas fabricantes não especificados. Esses casos foram preenchidos com uma string "Not available", representando que tais valores não foram especificados.

In [ ]:
from sklearn.impute import SimpleImputer

sub  = dados[['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios', 'anoLancamento']]
imp = SimpleImputer(missing_values=float('nan'), strategy='median')
imp.fit(sub)
completeData = pd.DataFrame(imp.transform(sub), columns=sub.columns, index=sub.index)
dados[['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios', 'anoLancamento']] = completeData[['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios', 'anoLancamento']]
print('Quantidade de NaNs apos SimpleInputer:')
print(dados.isnull().sum())

Quantidade de NaNs apos SimpleInputer:
nome                    0
plataforma              0
genero                  0
editora                 0
vendas                  0
avaliacao-criticos      0
numero-criticos         0
avaliacao-usuarios      0
numero-usuarios         0
fabricante            459
anoLancamento           0
dtype: int64


In [ ]:
dados['fabricante'] = dados['fabricante'].replace(float('nan'), 'Not available')

# Binarização de atributos nominais
As colunas de "plataforma" e "gênero" que originalmente são nominais/categóricas, foram binarizadas. Optou-se por fazer a binarização apenas dessas duas colunas, pois se realizada em todas as variáveis não númericas, a base de dados ficaria com muitos atributos (colunas) e teria o problema da "Maldição da Dimensionalidade"

In [ ]:
# sub = dados[['plataforma', 'genero', 'editora', 'fabricante']]
sub = dados[['plataforma', 'genero']]
completeData = pd.get_dummies(sub)
dados = pd.concat([dados, completeData], axis=1)
print(dados)

                                                   nome  ... genero_Strategy
0     Kawashima Ryuuta Kyouju Kanshuu Nouryoku Train...  ...               0
1                                            DT Carnage  ...               0
2                                           DeathSmiles  ...               0
3                                   Muv-Luv Alternative  ...               0
4                                                   MLB  ...               0
...                                                 ...  ...             ...
1412                                      Zumba Fitness  ...               0
1413                         Zumba Fitness: World Party  ...               0
1414                                          de Blob 2  ...               0
1415                                             nail'd  ...               0
1416                          pro evolution soccer 2011  ...               0

[1412 rows x 88 columns]


# Detecção de outliers

Não foi calculado outlier de ano de lançamento

Realizou-se então um reconhecimento/tratativa dos outliers. Foram utilizados dois métodos. O primeiro com a distância interquartílica, que acabou resultando em uma quantidade muito grande de outliers. Dessa forma, foi feita uma nova abordagem, utilizando o método do desvio padrão, que diminuiu bastante a quantidade de outliers encontrados.

In [ ]:
sub = dados[['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios']].copy()
q1 = sub.quantile(0.25)
q3 = sub.quantile(0.75)
iqr = q3 - q1

dfOutliers = ((sub < (q1 - 1.5*iqr)) | (sub > (q3 + 1.5*iqr)))
print('Quantidade de outliers (iqr): \n{}\n'.format(dfOutliers.sum()))

print('Quantidade de outliers (STD): \n{}\n'.format(sub[np.abs(sub-sub.mean()) <= 2*sub.std()].isnull().sum()))
mean = sub.mean()
std = sub.std()

outliers = sub[np.abs(sub-sub.mean()) >= 2*sub.std()].dropna(thresh=1)
display(outliers)
indexes = outliers.index.tolist()
# print(len(indexes))
for row in indexes:
    for col in outliers.columns:
        if not np.isnan(outliers.loc[row, col]):
            if outliers.loc[row, col] < mean[col]:
                sub.loc[row, col] = mean[col] - 1.9999*std[col]
            elif outliers.loc[row, col] > mean[col]:
                sub.loc[row, col] = mean[col] + 1.9999*std[col]
            else:
                print(row, col, outliers.loc[row, col])
# display(outliers)
dados[['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios']] = sub[['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios']]
print('Quantidade de outliers (STD): \n{}'.format(sub[np.abs(sub-mean) <= 2*std].isnull().sum()))

Quantidade de outliers (iqr): 
vendas                172
avaliacao-criticos    465
numero-criticos       130
avaliacao-usuarios    554
numero-usuarios       226
dtype: int64

Quantidade de outliers (STD): 
vendas                 45
avaliacao-criticos    104
numero-criticos        88
avaliacao-usuarios     75
numero-usuarios        37
dtype: int64



,vendas,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios
20,NaN,33.000000,NaN,4.7,NaN
24,NaN,39.000000,NaN,NaN,NaN
30,8.94,NaN,NaN,NaN,2605.0
32,27.25,NaN,NaN,NaN,NaN
53,NaN,43.000000,NaN,NaN,NaN
...,...,...,...,...,...
1392,NaN,NaN,140.0,NaN,NaN
1397,NaN,41.000000,NaN,NaN,NaN
1407,NaN,44.333333,NaN,NaN,NaN
1412,9.20,42.000000,NaN,NaN,NaN


Quantidade de outliers (STD): 
vendas                0
avaliacao-criticos    0
numero-criticos       0
avaliacao-usuarios    0
numero-usuarios       0
dtype: int64


# Normalização dos dados
Por fim, os dados númericos foram normalizados com base no maior valor, fazendo com que dessa forma todos os valores fiquem entre um intervalo de [0,1]

In [ ]:
from sklearn import preprocessing

sub = dados [['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios', 'anoLancamento']]
scaler = preprocessing.MinMaxScaler()
normalized = pd.DataFrame(scaler.fit_transform(sub), columns=sub.columns, index=sub.index)
dados[['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios', 'anoLancamento']] = normalized[['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios', 'anoLancamento']]
display(dados)


,nome,plataforma,genero,editora,vendas,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante,anoLancamento,plataforma_3DS,"plataforma_3DS, PS3","plataforma_3DS, PS3, PS4, PSP, PSV, Wii, X360, XOne","plataforma_3DS, PS3, PS4, PSV, Wii, WiiU, X360, XOne","plataforma_3DS, PS3, PS4, PSV, WiiU, X360, XOne","plataforma_3DS, PS3, PS4, Wii, WiiU, X360, XOne","plataforma_3DS, PS3, PSP","plataforma_3DS, PS3, PSP, PSV, Wii, WiiU, X360","plataforma_3DS, PS3, PSP, PSV, Wii, X360","plataforma_3DS, PS3, PSP, Wii, X360","plataforma_3DS, PS3, PSP, X360","plataforma_3DS, PS3, PSV, Wii, WiiU, X360","plataforma_3DS, PS3, PSV, Wii, X360","plataforma_3DS, PS3, PSV, WiiU, X360","plataforma_3DS, PS3, PSV, X360","plataforma_3DS, PS3, Wii, WiiU, X360","plataforma_3DS, PS3, Wii, X360","plataforma_3DS, PS3, WiiU, X360","plataforma_3DS, PS3, X360","plataforma_3DS, PSP","plataforma_3DS, PSP, PSV","plataforma_3DS, PSV","plataforma_3DS, Wii","plataforma_3DS, Wii, WiiU","plataforma_3DS, Wii, WiiU, X360","plataforma_3DS, WiiU",plataforma_PS3,"plataforma_PS3, PS4, PSV, WiiU, X360","plataforma_PS3, PS4, Wii, WiiU, X360, XOne",...,"plataforma_PS3, PSP, X360","plataforma_PS3, PSV","plataforma_PS3, PSV, Wii, WiiU, X360","plataforma_PS3, PSV, WiiU, X360","plataforma_PS3, PSV, X360","plataforma_PS3, Wii","plataforma_PS3, Wii, WiiU, X360","plataforma_PS3, Wii, X360","plataforma_PS3, WiiU","plataforma_PS3, WiiU, X360","plataforma_PS3, X360",plataforma_PS4,"plataforma_PS4, XOne",plataforma_PSP,"plataforma_PSP, PSV","plataforma_PSP, Wii","plataforma_PSP, Wii, X360","plataforma_PSP, X360",plataforma_PSV,"plataforma_PSV, Wii",plataforma_Wii,"plataforma_Wii, WiiU","plataforma_Wii, X360","plataforma_Wii, XOne",plataforma_WiiU,"plataforma_WiiU, X360",plataforma_X360,genero_Action,"genero_Action, Adventure",genero_Adventure,genero_Fighting,genero_Misc,genero_Platform,genero_Puzzle,genero_Racing,genero_Role-Playing,genero_Shooter,genero_Simulation,genero_Sports,genero_Strategy
0,Kawashima Ryuuta Kyouju Kanshuu Nouryoku Train...,PSP,Misc,Sega,0.013113,0.532591,0.354973,0.529996,0.035655,Not available,0.444444,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,DT Carnage,PSP,Racing,Agetec,0.004917,0.532591,0.354973,0.529996,0.035655,Axis Entertainment,0.444444,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,DeathSmiles,X360,Shooter,Rising Star Games,0.039338,0.672269,0.245100,0.860401,0.040607,Cave,0.555556,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Muv-Luv Alternative,PS3,Adventure,5pb,0.000000,0.532591,0.354973,0.529996,0.035655,Not available,0.444444,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,MLB,PSP,Sports,Sony Computer Entertainment,0.037699,0.765388,0.219745,0.907602,0.014856,989 Sports,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,Zumba Fitness,"PS3, Wii, X360",Sports,505 Games,1.000000,0.000000,0.084517,0.270392,0.012875,"Pipeworks Software, Inc.",0.555556,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1413,Zumba Fitness: World Party,"Wii, XOne",Misc,Majesco Entertainment,0.022947,0.602430,0.042259,0.411994,0.030703,Zoe Mode,0.888889,0,0,0